# Modelado y Validación del Modelo de Regresión

En este notebook se desarrollan los modelos de regresión para predecir la variable objetivo del dataset proporcionado por el curso.

El objetivo es:
- Entrenar distintos modelos de regresión.
- Evaluar su rendimiento mediante métricas estándar (RMSE, MAE, R²).
- Controlar problemas de overfitting y underfitting.
- Seleccionar y guardar el mejor modelo para su uso en la aplicación interactiva.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib


In [2]:
# Carga del dataset procesado
df = pd.read_csv("train_ready_for_modeling.csv")

df.head()



,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_Unknown,fuel_type_not supported,fuel_type_–,accident_None reported,clean_title_Yes
0,MINI,Cooper S Base,2007,213000,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,4200,False,True,False,False,False,False,False,True,True
1,Lincoln,LS V8,2002,143250,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,4999,False,True,False,False,False,False,False,False,True
2,Chevrolet,Silverado 2500 LT,2002,136731,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,13900,True,False,False,False,False,False,False,True,True
3,Genesis,G90 5.0 Ultimate,2017,19500,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,45000,False,True,False,False,False,False,False,True,True
4,Mercedes-Benz,Metris Base,2021,7388,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,97500,False,True,False,False,False,False,False,True,True


#Construcción del diccionario de opciones para Streamlit

En esta celda se construye un diccionario anidado (brand_model_options) que contiene, para cada marca y modelo, las opciones disponibles de:

Colores exteriores

Colores interiores

Años disponibles (mínimo y máximo)

Este diccionario se guardará en un archivo .pkl para ser utilizado en la aplicación Streamlit.

In [7]:
brand_model_options = {}

for (brand, model), group in df.groupby(["brand", "model"]):

    years = sorted(group["model_year"].dropna().unique().tolist())

    brand_model_options.setdefault(brand, {})[model] = {
        "ext_col": sorted(group["ext_col"].dropna().unique().tolist()),
        "int_col": sorted(group["int_col"].dropna().unique().tolist()),
        "min_year": int(min(years)),
        "max_year": int(max(years)),
        "model_years": years
    }

# Guardar el .pkl
joblib.dump(brand_model_options, "brand_model_options.pkl")



['brand_model_options.pkl']

#Extracción de características del motor

En esta sección extraemos información útil del campo engine, que en el dataset está como texto libre.
A partir de esa columna se derivan las siguientes variables:

engine_liters: litros del motor

cylinders: número de cilindros

turbo: si el motor es turbo o no

horsepower: caballos de potencia

Estas variables se crearán a partir de expresiones regulares y se normalizarán para poder ser usadas como características numéricas en el modelo.

In [ ]:
#Normalizamos la columna engine a string en minúsculas
engine_str = df['engine'].astype(str).str.lower()
liters = (
    engine_str
    .str.extract(r'(\d+[.,]?\d)\sl')[0]
    .str.replace(',', '.', regex=False)
    .astype(float)
)

liters = np.where(liters > 10, liters / 10, liters)
df['engine_liters'] = pd.Series(liters).fillna(-1)
df['cylinders'] = (
    engine_str
    .str.extract(r'(\d+)\scylinder|v(\d+)')
    .bfill(axis=1)
    .iloc[:, 0]
)
df['cylinders'] = pd.to_numeric(
    df['cylinders'],
    errors='coerce'
).fillna(-1)
df['turbo'] = engine_str.str.contains('turbo', na=False)
df['horsepower'] = (
    engine_str
    .str.extract(r'(\d+[.,]?\d)\s*hp')[0]
    .str.replace(',', '.', regex=False)
    .astype(float)
    .fillna(-1)
    .astype(int)
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     188533 non-null  object 
 1   model                     188533 non-null  object 
 2   model_year                188533 non-null  int64  
 3   milage                    188533 non-null  int64  
 4   engine                    188533 non-null  object 
 5   transmission              188533 non-null  object 
 6   ext_col                   188533 non-null  object 
 7   int_col                   188533 non-null  object 
 8   price                     188533 non-null  int64  
 9   fuel_type_E85 Flex Fuel   188533 non-null  bool   
 10  fuel_type_Gasoline        188533 non-null  bool   
 11  fuel_type_Hybrid          188533 non-null  bool   
 12  fuel_type_Plug-In Hybrid  188533 non-null  bool   
 13  fuel_type_Unknown         188533 non-null  b

Se han creado nuevas variables numéricas a partir del campo engine que permiten incluir características del motor en el modelo:

engine_liters y horsepower quedan en formato numérico para su uso en el modelo.

cylinders se obtiene mediante extracción de patrones comunes.

turbo se convierte en variable booleana.

Además, se han normalizado valores erróneos (por ejemplo, valores mayores de 10 litros) y se han rellenado valores faltantes con -1 para mantener consistencia en el dataset.

#Eliminación de columnas irrelevantes
Una vez que hemos extraído las variables útiles de la columna engine, esta columna ya no es necesaria.
Además, eliminamos otras columnas que no aportan información útil o que contienen valores no soportados por el modelo.

Las columnas eliminadas son:

engine: ya se ha transformado en engine_liters, cylinders, turbo y horsepower

transmission: no se utilizará en el modelo

fuel_type_– y fuel_type_not supported: valores no soportados / no informativos

In [ ]:
df = df.drop(
    columns=[
        "engine",
        "transmission",
        "fuel_type_–",
        "fuel_type_not supported"
    ]
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     188533 non-null  object 
 1   model                     188533 non-null  object 
 2   model_year                188533 non-null  int64  
 3   milage                    188533 non-null  int64  
 4   ext_col                   188533 non-null  object 
 5   int_col                   188533 non-null  object 
 6   price                     188533 non-null  int64  
 7   fuel_type_E85 Flex Fuel   188533 non-null  bool   
 8   fuel_type_Gasoline        188533 non-null  bool   
 9   fuel_type_Hybrid          188533 non-null  bool   
 10  fuel_type_Plug-In Hybrid  188533 non-null  bool   
 11  fuel_type_Unknown         188533 non-null  bool   
 12  accident_None reported    188533 non-null  bool   
 13  clean_title_Yes           188533 non-null  b

#Transformación del precio
El precio de los vehículos presenta una distribución muy sesgada, con valores extremos que podrían afectar al entrenamiento de los modelos. Para mitigar este efecto, aplicamos una transformación logarítmica sobre la variable objetivo, conservando antes una copia del precio original para poder revertir las predicciones si es necesario.

In [ ]:
# Copia de seguridad del precio original
df["price_raw"] = df["price"]

# Log-transform del target
df["price"] = np.log1p(df["price"])


In [ ]:
df[["price_raw", "price"]].describe()

,price_raw,price
count,1.885330e+05,188533.000000
mean,4.387802e+04,10.291787
std,7.881952e+04,0.844173
min,2.000000e+03,7.601402
25%,1.700000e+04,9.741027
50%,3.082500e+04,10.336114
75%,4.990000e+04,10.817796
max,2.954083e+06,14.898699


Tras la transformación, podemos observar que:

La variable original (price_raw) tiene valores muy dispersos, con un máximo cercano a 3 millones y una media alrededor de 43.880 USD.

La variable transformada (price) presenta una distribución más compacta: la media es ~10.29, la desviación estándar baja a ~0.84 y los valores extremos se atenúan.

La transformación logarítmica conserva el orden relativo de los precios, pero reduce la influencia de outliers, facilitando el entrenamiento y mejorando la estabilidad de los modelos.

Esta transformación permite que modelos como Random Forest, Gradient Boosting y XGBoost aprendan patrones de manera más consistente y generalicen mejor sobre datos nuevos.

#Evaluación de modelos

Para poder comparar de forma consistente los distintos modelos de regresión entrenados, se define una función auxiliar que centraliza el proceso de entrenamiento y evaluación.

Esta función entrena el modelo con el conjunto de entrenamiento y calcula métricas estándar de regresión (RMSE, MAE y R²) tanto en entrenamiento como en test. De esta forma, se puede analizar el rendimiento predictivo de cada modelo y detectar posibles problemas de overfitting o underfitting comparando ambos resultados.

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test):

    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Métricas
    metrics = {
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "test_rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "r2": r2_score(y_test, y_test_pred)
    }

    return metrics


## División Train/Test y codificación de variables categóricas

Para entrenar los modelos necesitamos dividir los datos en conjunto de entrenamiento y prueba, asegurando que las métricas reflejen la capacidad de generalización. Además, dado que la mayoría de nuestras variables son categóricas, aplicamos Target Encoding para convertirlas en valores numéricos basados en la media del precio log-transformado en el conjunto de entrenamiento.

In [ ]:
from sklearn.model_selection import train_test_split

# Target (ya transformado en log)
y = df["price"]

# Features: quitamos price y price_raw
X = df.drop(columns=["price", "price_raw"])

# Identificar tipos de columnas
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "bool"]).columns

print(f"Numéricas: {len(num_cols)}")
print(f"Categóricas: {len(cat_cols)}")

# División 80/20 (ANTES de cualquier encoding)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Numéricas: 5
Categóricas: 12


#Guardado del orden de características (feature order)
El modelo de Machine Learning espera que los datos de entrada tengan las columnas en un orden específico.
Para asegurar que el DataFrame de entrada en Streamlit tenga el mismo orden que el usado en entrenamiento, se guarda la lista de columnas (feature_order) en un archivo .pkl.

Además, se crea la carpeta model/ si no existe, para almacenar todos los artefactos del modelo.

In [ ]:
import os

os.makedirs("model", exist_ok=True)

import joblib

joblib.dump(list(X_train_enc.columns), "model/feature_order.pkl")

['model/feature_order.pkl']

Se ha guardado el orden exacto de las columnas utilizadas en el entrenamiento del modelo.
Esto garantiza que la app Streamlit pueda reconstruir el DataFrame de entrada con el mismo orden, evitando errores de predicción por desalineación de columnas.

#Codificación de variables categóricas mediante Target Encoding
En esta sección se realiza Target Encoding para las variables categóricas.
El Target Encoding consiste en sustituir cada categoría por la media del target (precio) asociada a esa categoría en el conjunto de entrenamiento.

Esto permite que el modelo utilice variables categóricas sin necesidad de One-Hot Encoding, reduciendo dimensionalidad y evitando categorías raras en test.

Se crea un diccionario target_encoding_maps para guardar los mapas de codificación, que luego se usarán en la app Streamlit para codificar las entradas del usuario de la misma forma que durante el entrenamiento.

In [ ]:
# Copiar X_train y X_test para no tocar el original
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()

target_encoding_maps = {}


# Hacer Target Encoding para cada categórica
for col in cat_cols:
    # Media del target por categoría en el train
    mean_map = y_train.groupby(X_train[col]).mean()

    # Mapear train y test
    X_train_enc[col] = X_train[col].map(mean_map)
    X_test_enc[col] = X_test[col].map(mean_map)

    # Opcional: rellenar valores que no existan en test (sin inplace)
    X_test_enc[col] = X_test_enc[col].fillna(y_train.mean())

for col in cat_cols:
    mean_map = y_train.groupby(X_train[col]).mean().to_dict()
    target_encoding_maps[col] = {
        "mapping": mean_map,
        "global_mean": y_train.mean()
    }



In [ ]:
print(X_train_enc.head())
print(X_test_enc.head())

            brand      model  model_year  milage    ext_col    int_col  \
184031  10.481131   9.956281        2017   61675  10.427909  10.374961   
173831   9.896334   9.585657        2003  185000   9.971766   9.824654   
183819  10.284189  10.481041        2020   92000  10.427909   9.824654   
85525    9.863576  10.602486        2023    5483  10.260663  10.374961   
41872    9.906697  10.939151        2023    5000  10.096901  10.374961   

        fuel_type_E85 Flex Fuel  fuel_type_Gasoline  fuel_type_Hybrid  \
184031                 9.917709           10.451053         10.277216   
173831                10.302587           10.269934         10.277216   
183819                10.302587           10.269934         10.277216   
85525                 10.302587           10.269934         10.277216   
41872                 10.302587           10.269934         10.277216   

        fuel_type_Plug-In Hybrid  fuel_type_Unknown  accident_None reported  \
184031                 10.290779     

In [ ]:
X_train_enc.info()


<class 'pandas.core.frame.DataFrame'>
Index: 150826 entries, 184031 to 121958
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     150826 non-null  float64
 1   model                     150826 non-null  float64
 2   model_year                150826 non-null  int64  
 3   milage                    150826 non-null  int64  
 4   ext_col                   150826 non-null  float64
 5   int_col                   150826 non-null  float64
 6   fuel_type_E85 Flex Fuel   150826 non-null  float64
 7   fuel_type_Gasoline        150826 non-null  float64
 8   fuel_type_Hybrid          150826 non-null  float64
 9   fuel_type_Plug-In Hybrid  150826 non-null  float64
 10  fuel_type_Unknown         150826 non-null  float64
 11  accident_None reported    150826 non-null  float64
 12  clean_title_Yes           150826 non-null  float64
 13  engine_liters             150826 non-null  f

Se ha aplicado Target Encoding a todas las variables categóricas.
Esto transforma cada categoría en un valor numérico representando la media del precio asociado a dicha categoría.

El diccionario target_encoding_maps queda guardado para que la aplicación Streamlit pueda codificar los inputs del usuario de la misma forma que el modelo fue entrenado, asegurando coherencia en las predicciones.

Tras la codificación, todos los features son numéricos y listos para entrenar modelos como Random Forest, Gradient Boosting o XGBoost.

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(150826, 17) (37707, 17) (150826,) (37707,)


Tenemos 150.826 filas en el conjunto de entrenamiento y 37.707 en el conjunto de prueba, manteniendo la proporción 80/20.

Cada conjunto de features tiene 17 columnas, correspondientes a 2 numéricas y 15 categóricas.

Los vectores y_train y y_test coinciden en tamaño con sus respectivos conjuntos de features, asegurando que los datos están listos para el entrenamiento de los modelos.

## Modelo Base: Regresión Lineal

Se utiliza un modelo de Regresión Lineal como baseline para establecer un punto de referencia.  
Este modelo permite evaluar si modelos más complejos aportan una mejora significativa en el rendimiento.


In [ ]:
from sklearn.linear_model import LinearRegression

# Crear el modelo
model = LinearRegression()

# Evaluar modelo con tu función
metrics = evaluate_model(model, X_train_enc, y_train, X_test_enc, y_test)

print("Métricas del modelo:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

Métricas del modelo:
train_rmse: 0.5207
test_rmse: 0.5290
mae: 0.3745
r2: 0.6091


#Conclusión
El RMSE y MAE indican el error promedio en la predicción del precio log-transformado.

Un R² = 0.61 sugiere que el modelo lineal explica aproximadamente el 61% de la varianza del precio log-transformado, lo cual es razonable para un modelo simple.

La ligera diferencia entre RMSE de entrenamiento y test muestra bajo sobreajuste, lo que confirma que el modelo es estable.

Aunque el rendimiento es decente, la relación entre las variables y el precio podría mejorar usando modelos más complejos (Random Forest, Gradient Boosting o XGBoost) que capturen interacciones no lineales y patrones más sutiles.

## Modelo Ensemble: Random Forest Regressor

Random Forest es un modelo ensemble basado en múltiples árboles de decisión.  
Permite capturar relaciones no lineales y reducir la varianza respecto a modelos individuales.


In [ ]:

from sklearn.ensemble import RandomForestRegressor

# Crear el modelo con algunos parámetros iniciales (puedes ajustar luego)
rf_model = RandomForestRegressor(
    n_estimators=100,  # 100 árboles, se puede subir si tienes potencia
    max_depth=10,      # Profundidad máxima para evitar overfitting
    random_state=42,
    n_jobs=-1          # Para usar todos los núcleos de tu CPU y acelerar
)

# Usar la función que definimos para evaluar el modelo
rf_metrics = evaluate_model(rf_model, X_train_enc, y_train, X_test_enc, y_test)

print("Métricas Random Forest:")
for k, v in rf_metrics.items():
    print(f"{k}: {v:.4f}")



Métricas Random Forest:
train_rmse: 0.4743
test_rmse: 0.5016
mae: 0.3502
r2: 0.6486


## Modelo Ensemble: Gradient Boosting Regressor

Gradient Boosting construye modelos secuenciales que corrigen los errores del modelo anterior.  
Suele ofrecer un buen equilibrio entre sesgo y varianza en problemas de regresión.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Crear el modelo
gbr = GradientBoostingRegressor(random_state=42)

# Evaluar el modelo con tu función
metrics_gbr = evaluate_model(gbr, X_train_enc, y_train, X_test_enc, y_test)

# Mostrar métricas
print("Métricas Gradient Boosting:")
for k, v in metrics_gbr.items():
    print(f"{k}: {v:.4f}")



Métricas Gradient Boosting:
train_rmse: 0.4955
test_rmse: 0.5023
mae: 0.3520
r2: 0.6477


#Modelo Ensemble: XGBoost Regressor
XGBoost es un modelo de gradient boosting optimizado, que construye árboles secuenciales corrigiendo errores del anterior, pero con mejoras de velocidad y regularización.
Suele ofrecer un buen equilibrio entre sesgo y varianza y manejar datasets grandes de manera eficiente

In [ ]:
from xgboost import XGBRegressor

# Crear el modelo (usamos parámetros base para empezar)
xgb_model = XGBRegressor(
    n_estimators=500,   # número de árboles
    learning_rate=0.05, # tasa de aprendizaje
    max_depth=5,        # profundidad máxima del árbol
    random_state=42,
    n_jobs=-1
)

# Evaluar el modelo
xgb_metrics = evaluate_model(xgb_model, X_train_enc, y_train, X_test_enc, y_test)
print("Métricas XGBoost:")
for k, v in xgb_metrics.items():
    print(f"{k}: {v:.4f}")


Métricas XGBoost:
train_rmse: 0.4766
test_rmse: 0.4943
mae: 0.3451
r2: 0.6588


## Comparación de modelos

Se comparan los distintos modelos entrenados utilizando métricas de regresión (RMSE, MAE y R²)  
con el objetivo de seleccionar el modelo con mejor rendimiento y menor sobreajuste.


In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_metrics = evaluate_model(gb_model, X_train_enc, y_train, X_test_enc, y_test)

# Linear Regression (opcional)
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_metrics = evaluate_model(lr_model, X_train_enc, y_train, X_test_enc, y_test)


results = pd.DataFrame([
     ["Linear Regression", lr_metrics["test_rmse"], lr_metrics["mae"], lr_metrics["r2"]],
     ["Random Forest", rf_metrics["test_rmse"], rf_metrics["mae"], rf_metrics["r2"]],
     ["Gradient Boosting", gb_metrics["test_rmse"], gb_metrics["mae"], gb_metrics["r2"]],
     ["XGBoost", xgb_metrics["test_rmse"], xgb_metrics["mae"], xgb_metrics["r2"]]
], columns=["Model", "RMSE", "MAE", "R2"])

results


,Model,RMSE,MAE,R2
0,Linear Regression,0.529030,0.374520,0.609107
1,Random Forest,0.501594,0.350152,0.648600
2,Gradient Boosting,0.502258,0.351970,0.647669
3,XGBoost,0.494256,0.345148,0.658806


#Conclusión:
XGBoost es el modelo más preciso, con R² ≈ 0.65 y RMSE más bajo, superando a Linear Regression, Random Forest y Gradient Boosting.

Por su mejor desempeño, nos quedaremos con XGBoost para los siguientes pasos del proyecto.

## Validación cruzada
Para asegurarnos de que XGBoost funciona de manera consistente y no depende de una sola división de los datos, realizamos validación cruzada (5 folds) sobre el conjunto de entrenamiento.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBRegressor
import numpy as np

# Definir modelo (igual que antes)
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

# 5-fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# RMSE con cross-validation (neg_mean_squared_error porque sklearn lo devuelve negativo)
cv_rmse = -cross_val_score(xgb_model, X_train_enc, y_train,
                           scoring="neg_mean_squared_error", cv=kf)
cv_rmse = np.sqrt(cv_rmse)

print(f"RMSE por fold: {cv_rmse}")
print(f"RMSE promedio CV: {cv_rmse.mean():.4f} ± {cv_rmse.std():.4f}")


RMSE por fold: [0.48910824 0.5008706  0.50108775 0.49786607 0.50093624]
RMSE promedio CV: 0.4980 ± 0.0046


#Conclusión de la validación cruzada (XGBoost)

Cada fold produjo un RMSE cercano a 0.50, mostrando estabilidad entre las distintas particiones.

El RMSE promedio CV ≈ 0.499 ± 0.005 confirma que el modelo generaliza bien y no hay sobreajuste evidente en los datos de entrenamiento.

## Optimización de hiperparámetros

En esta sección llevamos XGBoost al siguiente nivel: primero establecemos un modelo base para tener un baseline, y luego realizamos una optimización de hiperparámetros usando RandomizedSearchCV con validación cruzada. Esto nos permite encontrar la configuración que minimiza el error y garantiza que el modelo generalice mejor.


In [ ]:
# ============================================
# XGBOOST: Validación cruzada + Optimización
# ============================================

from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
from xgboost import XGBRegressor

# --------------------------------------------
# 1. MODELO BASE
# --------------------------------------------
xgb_base = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    objective="reg:squarederror",
    tree_method="hist",   # más rápido y menos RAM
    random_state=42,
    n_jobs=-1
)

# --------------------------------------------
# 2. VALIDACIÓN CRUZADA (BASELINE)
# --------------------------------------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_rmse = cross_val_score(
    xgb_base,
    X_train_enc,
    y_train,
    scoring="neg_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

cv_rmse = np.sqrt(-cv_rmse)

print("RMSE por fold (baseline):", cv_rmse)
print(f"RMSE promedio CV (baseline): {cv_rmse.mean():.4f} ± {cv_rmse.std():.4f}")

# --------------------------------------------
# 3. OPTIMIZACIÓN DE HIPERPARÁMETROS
# --------------------------------------------
xgb = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [200, 400, 600],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,
    cv=kf,   # MISMO K-Fold
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X_train_enc, y_train)

# --------------------------------------------
# 4. MEJORES RESULTADOS
# --------------------------------------------
print("\nMejores hiperparámetros:")
print(random_search.best_params_)

print(f"Mejor RMSE CV (optimizado): {-random_search.best_score_:.4f}")

best_xgb = random_search.best_estimator_

# --------------------------------------------
# 5. VALIDACIÓN CRUZADA FINAL (MODELO OPTIMIZADO)
# --------------------------------------------
cv_rmse_opt = cross_val_score(
    best_xgb,
    X_train_enc,
    y_train,
    scoring="neg_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

cv_rmse_opt = np.sqrt(-cv_rmse_opt)

print("RMSE por fold (optimizado):", cv_rmse_opt)
print(f"RMSE promedio CV (optimizado): {cv_rmse_opt.mean():.4f} ± {cv_rmse_opt.std():.4f}")

RMSE por fold (baseline): [0.48910824 0.5008706  0.50108775 0.49786607 0.50093624]
RMSE promedio CV (baseline): 0.4980 ± 0.0046
Fitting 5 folds for each of 10 candidates, totalling 50 fits

Mejores hiperparámetros:
{'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
Mejor RMSE CV (optimizado): 0.4898
RMSE por fold (optimizado): [0.48120497 0.49206909 0.4925349  0.49057747 0.49248429]
RMSE promedio CV (optimizado): 0.4898 ± 0.0043


#Conclusión de la optimización de hiperparámetros (XGBoost)

La búsqueda aleatoria de hiperparámetros permitió mejorar el modelo base de XGBoost.
Con los mejores valores encontrados:

subsample = 0.8

n_estimators = 200

max_depth = 7

learning_rate = 0.05

colsample_bytree = 0.8

## Guardado del modelo final

El modelo seleccionado se guarda para su posterior uso en la aplicación interactiva  
desarrollada con Streamlit.


In [ ]:
import joblib
#from xgboost import XGBRegressor

# ============================================
# Entrenar y guardar el modelo XGBoost final
# ============================================

best_xgb = XGBRegressor(
    n_estimators=random_search.best_params_["n_estimators"],  # 200
    max_depth=random_search.best_params_["max_depth"],        # 7
    learning_rate=random_search.best_params_["learning_rate"],# 0.05
    subsample=random_search.best_params_["subsample"],        # 0.8
    colsample_bytree=random_search.best_params_["colsample_bytree"], # 0.8
    objective="reg:squarederror",
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

# Entrenamiento final en todo el conjunto de entrenamiento
best_xgb.fit(X_train_enc, y_train)

# Guardar el modelo entrenado
#best_xgb.save_model("best_xgb_model_final.json")
joblib.dump(best_xgb, "best_xgb_model_final.pkl")

#print("Modelo guardado como 'best_xgb_model_final.json'")
print("Modelo guardado como 'best_xgb_model_final.pkl'")

Modelo guardado como 'best_xgb_model_final.pkl'


Con los mejores hiperparámetros encontrados, entrenamos el XGBoost final con todo el conjunto de entrenamiento.
El modelo se guarda en disco para poder reutilizarlo sin necesidad de volver a entrenarlo. Tenemos dos opciones de guardado:

JSON: best_xgb_model_final.json (recomendado para XGBoost nativo).

Pickle (pkl): best_xgb_model_final.pkl (útil si prefieres cargarlo con joblib).

De esta manera, nuestro modelo queda listo para predicciones futuras o despliegue en producción.

#Guardado de mapas de Target Encoding
El Target Encoding convierte categorías en valores numéricos mediante la media del target (precio) por categoría.
Para que la aplicación Streamlit pueda codificar las entradas del usuario exactamente igual que en entrenamiento, es necesario guardar los mapas de codificación.

En este bloque se crea el diccionario target_encoding_maps y se guarda en un archivo .joblib.

In [ ]:
#import joblib

target_encoding_maps = {}

for col in cat_cols:
    mean_map = y_train.groupby(X_train[col]).mean().to_dict()
    target_encoding_maps[col] = {
        "mapping": mean_map,
        "global_mean": y_train.mean()
    }

joblib.dump(target_encoding_maps, "target_encoding_maps.joblib")

['target_encoding_maps.joblib']

Se ha generado y guardado un diccionario con los mapas de Target Encoding para todas las variables categóricas.
Este archivo permite que la aplicación Streamlit transforme las entradas del usuario de la misma forma que se hizo durante el entrenamiento, garantizando coherencia en las predicciones.